In [1]:
import os
import sys
module_path = os.path.abspath(os.path.join('..'))
if module_path not in sys.path:
    sys.path.append(module_path)
    
from main import TradingBot

tb = TradingBot()

In [3]:
from jugaad_data.nse import NSELive

quote = NSELive().stock_quote('JSWSTEEL')
info = quote['priceInfo']
date = quote['metadata']['lastUpdateTime']
date

'14-Jul-2023 16:00:00'

In [14]:
import pandas as pd
pd.to_datetime(date).date()

datetime.date(2023, 7, 14)

In [12]:
NSELive().all_indices()['timestamp']

'14-Jul-2023 15:30:00'

In [16]:
NSELive().live_index('NIFTY 50')

{'name': 'NIFTY 50',
 'advance': {'declines': '16', 'advances': '34', 'unchanged': '0'},
 'timestamp': '14-Jul-2023 16:00:00',
 'data': [{'priority': 1,
   'symbol': 'NIFTY 50',
   'identifier': 'NIFTY 50',
   'open': 19493.45,
   'dayHigh': 19595.35,
   'dayLow': 19433.5,
   'lastPrice': 19564.5,
   'previousClose': 19413.75,
   'change': 150.75,
   'pChange': 0.78,
   'ffmc': 742978397.23,
   'yearHigh': 19595.35,
   'yearLow': 16438.75,
   'totalTradedVolume': 291165841,
   'totalTradedValue': 254843274666.57,
   'lastUpdateTime': '14-Jul-2023 16:00:00',
   'nearWKH': 0.15743530990770027,
   'nearWKL': -19.014523610371835,
   'perChange365d': 22.75,
   'date365dAgo': '14-Jul-2022',
   'chart365dPath': 'https://static.nseindia.com/sparklines/365d/NIFTY-50.jpg',
   'date30dAgo': '14-Jun-2023',
   'perChange30d': 4.31,
   'chart30dPath': 'https://static.nseindia.com/sparklines/30d/NIFTY-50.jpg',
   'chartTodayPath': 'https://static.nseindia.com/sparklines/today/NIFTY-50.jpg'},
  {'prio